In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd 

def objective(trial: optuna.Trial,
              xtrain: pd.DataFrame,
              ytrain: pd.DataFrame,
              xtest: pd.DataFrame,
              ytest: pd.DataFrame) -> float:
    
    # Define hyperparameters to tune
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 5, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1e2),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 1e-3, 1e3),
        'custom_metric': ['AUC'],
        'eval_metric': 'AUC',
        'random_state': 42
    }
    
    # Initialize CatBoostClassifier with trial parameters
    model = CatBoostClassifier(**params, silent=True)
    
    # Train the model
    model.fit(xtrain, ytrain, eval_set=(xtest, ytest), use_best_model=True)
    
    # Predict and evaluate
    y_pred_proba = model.predict_proba(xtest)
    roc_auc = roc_auc_score(ytest, y_pred_proba)
    
    # Return the metric to optimize (negative accuracy for maximization)
    return roc_auc

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')  # Minimizing the negative accuracy
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print('Best hyperparameters:', study.best_params)
print('Best score:', study.best_value)